In [ ]:
import databento as db
import pandas as pd
import os
from datetime import datetime,timedelta,time
import math
import numpy as np
import csv

In [ ]:
selected_columns = [
    'ts_event', 'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00',
    'bid_sz_01', 'ask_sz_01', 'bid_sz_02', 'ask_sz_02',
    'bid_sz_03', 'ask_sz_03', 'bid_sz_04', 'ask_sz_04',
    'bid_sz_05', 'ask_sz_05', 'bid_sz_06', 'ask_sz_06',
    'bid_sz_07', 'ask_sz_07', 'bid_sz_08', 'ask_sz_08',
    'bid_sz_09', 'ask_sz_09'
]

directory = r"insert path to directory containing databento files"

dataset = pd.DataFrame(columns=selected_columns)


for filename in os.listdir(directory):
    if filename.endswith(".dbn.zst"):
        file_path = os.path.join(directory, filename)
        try:
            stored_data = db.DBNStore.from_file(file_path)
            print(f"Reading DBN file: {filename}")
            df = stored_data.to_df()
        
            filtered_df = df[df['symbol'] == 'ESZ3']
            dataset1 = filtered_df[selected_columns]
            dataset = dataset.append(dataset1)
        except Exception as e:
            print(f"ERROR while processing {filename}: {str(e)}")


print(dataset.shape)



In [ ]:
print(dataset.shape)

In [ ]:
bids_columns = ['bid_sz_00', 'bid_sz_01', 'bid_sz_02', 'bid_sz_03', 'bid_sz_04', 'bid_sz_05', 'bid_sz_06', 'bid_sz_07', 'bid_sz_08', 'bid_sz_09']
asks_columns = ['ask_sz_00', 'ask_sz_01', 'ask_sz_02', 'ask_sz_03', 'ask_sz_04', 'ask_sz_05', 'ask_sz_06', 'ask_sz_07', 'ask_sz_08', 'ask_sz_09']

dataset['bids'] = dataset[bids_columns].sum(axis=1)
dataset['asks'] = dataset[asks_columns].sum(axis=1)


dataset['Skew'] = np.where((dataset['bids'] > 0) & (dataset['asks'] > 0),
                           np.log(dataset['bids']) - np.log(dataset['asks']),
                           0)


dataset = dataset.drop(['bids', 'asks'], axis=1)


In [ ]:
print(dataset.shape)
print(dataset['Skew'].isnull().sum())

In [ ]:
fileout = 'TradingLogTest.csv'
with open(fileout, 'r', newline='') as file:
    reader = csv.DictReader(file)
    headers = reader.fieldnames
print(headers)

In [ ]:

positions = 0
totalprofit=0
commission_cost=0

shortexitprofit=0
shortexitloss=0
longexitprofit=0
longexitloss=0
short_list=[]
long_list=[] 
short_price_list=[]
long_price_list=[]
exitpricelist=[]
profitlist=[]
running_dict={}
k=0.3
spread=0.25

start_hold=None
end_hold=None
interval_start_time=None
interval_end_time=None


profits=0
losses=0


max_positions=20
stop_loss=float('inf')
profit_target=0.25
start_time=time(9,30,0)
end_time=time(16,30,0)


def long(entryprice):
    global positions,commission_cost,long_list,long_price_list,spread
    if len(long_list)>0:
        d=entryprice-long_list[len(long_list)-1]
        if d==spread:
            price=entryprice*2
            positions += 2
            commission_cost+=2.5
            long_list.append(price)
            long_price_list.append(entryprice)
            print("LONG | skew:", skew, "| positions:", positions,"PRICE:",entryprice)
            print("RunningPNL:",calcrunningpnl(positions,dataset['bid_px_00'][i]))
    else:
        price=entryprice*2
        positions += 2
        commission_cost+=2.5
        long_list.append(price)
        long_price_list.append(entryprice)
        print("LONG | skew:", skew, "| positions:", positions,"PRICE:",entryprice)
        print("RunningPNL:",calcrunningpnl(positions,dataset['bid_px_00'][i]))

    
        keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','Skew']
        for a in range(1,(abs(positions)//2)+1):
            keys.append(f'EntryPrice{a}')
        for b in range(1,(abs(positions)//2)+1):
            keys.append(f'RunningPnL{b}')
        values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],1,positions,0,skew]
        for a in long_price_list:
            values.append(a)
        for b in range(1,(abs(positions)//2)+1):
            values.append(calcrunningpnl(positions,dataset['bid_px_00'][i])*2)
        row=dict(zip(keys,values))
        row = {header: row.get(header, '') for header in headers}
        with open(fileout, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            file.seek(0,2)
            writer.writerow(row)
        

def short(entryprice):
    global positions,commission_cost,short_list,short_price_list,spread
    if len(short_list)>0:
        d=short_list[len(short_list)-1]-entryprice
        if d ==spread:
            price=entryprice*2
            short_list.append(price)
            short_price_list.append(entryprice)
            positions -= 2
            commission_cost+=2.5
            print("SHORT | skew:", skew, "| positions:", positions,"PRICE:",entryprice)
            print("RunningPNL:",calcrunningpnl(positions,dataset['ask_px_00'][i]))
    else:
        price=entryprice*2
        short_list.append(price)
        short_price_list.append(entryprice)
        positions -= 2
        commission_cost+=2.5
        print("SHORT | skew:", skew, "| positions:", positions,"PRICE:",entryprice)
        print("RunningPNL:",calcrunningpnl(positions,dataset['ask_px_00'][i]))
        
        keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','Skew']
        for a in range(1,(abs(positions)//2)+1):
            keys.append(f'EntryPrice{a}')
        for b in range(1,(abs(positions)//2)+1):
            keys.append(f'RunningPnL{b}')
        values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],-1,positions,0,skew]
        for a in short_price_list:
            values.append(a)
        for b in range(1,(abs(positions)//2)+1):
            values.append(calcrunningpnl(positions,dataset['ask_px_00'][i])*2)
        row=dict(zip(keys,values))
        row = {header: row.get(header, '') for header in headers}
        with open(fileout, 'a', newline='') as file:
            writer = csv.DictWriter(file, fieldnames=headers)
            file.seek(0,2)
            writer.writerow(row)
    

def exitlong(entryprice):
    global positions,commission_cost,long_list,long_price_list
    global longexitloss, longexitprofit
    
    price=entryprice*2
    positions -= 2
    commission_cost+=2.5
    diff=price-long_list[len(long_list)-1]
    long_list.pop()
    long_price_list.pop()
    if diff>0:
        sig='Profit'        
        longexitprofit+=abs(diff)
                
    else:
        sig='Loss'
        longexitloss+=abs(diff)
    print("EXITLONG | skew:", skew,"|","diff:",diff, "| positions:", positions,"| Exit Price:", entryprice)
    keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','PnL','Skew']
    
    for a in range(1,len(long_price_list)+1):
        keys.append(f'EntryPrice{a}')
    for a in range(1,len(long_price_list)+1):
        keys.append(f'RunningPnL{a}')
    
    
    values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],0,positions,1,sig,skew]
    for b in long_price_list:
        values.append(b)
    for b in range(len(long_price_list)):
        values.append(calcrunningpnl(positions,dataset['bid_px_00'][i])*2)
    
    row=dict(zip(keys,values))
    
    row = {header: row.get(header, '') for header in headers}
    
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row)
    

def exitshort(entryprice):
    global positions,commission_cost,short_list,short_price_list
    global df
    global shortexitloss, shortexitprofit
    price=entryprice*2
    positions+=2
    diff=(short_list[len(short_list)-1]-price)
    if diff>0:
        sig='Profit'
        shortexitprofit+=diff
    else:
        sig='Loss'
        shortexitloss+=abs(diff)
    short_list.pop()
    short_price_list.pop()
    commission_cost+=2.5
    print("EXITSHORT","|","Diff:",diff,"Skew:",skew,"|","Positions:",positions,"| Exit Price:",entryprice)
    keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','PnL','Skew']
    for a in range(1,len(short_price_list)+1):
        keys.append(f'EntryPrice{a}')
    for a in range(1,len(short_price_list)+1):
        keys.append(f'RunningPnL{a}')
    
    values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],0,positions,1,sig,skew]
    for b in short_price_list:
        values.append(b)
    for b in range (len(short_price_list)):
        values.append(calcrunningpnl(positions,dataset['ask_px_00'][i])*2)
    
    row=dict(zip(keys,values))
   
    row = {header: row.get(header, '') for header in headers}
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row)
    

def closelongpositions(exitprice,positions,sign):
    global long_list,commission_cost,longexitloss,longexitprofit,totalprofit,df
    
    price=exitprice*2
    commission_cost+=2.5
    print(f"Closing {positions} long positions | Exit Price:", exitprice,"i:",i)
    
    
    for j in range(len(long_list)):
        
        diff=price-long_list[j]
        if diff>0:
            
            longexitprofit+=abs(diff)
        elif diff<0:
            longexitloss+=abs(diff)
    x=calcexitpnl(positions,exitprice)
    if x>0:
        sig='Profit'
    else:
        sig='Loss'
    
    keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','PnL','Skew']
    for a in range(1,11):
        keys.append(f'RunningPnL{a}')
   
    values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],0,positions,1,sig,skew]
    for b in range(1,11):
        values.append((calcrunningpnl(positions,exitprice)+0.125)*2)
    if sign==1:
        keys.append('ProfitTarget')
        values.append(1)
    elif sign==-1:
        keys.append('StopLoss')
        values.append(1)
   
    row=dict(zip(keys,values))
    row = {header: row.get(header, '') for header in headers}
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row)
   
    
    exitpricelist.append([exitprice,"LONG"])
    profitlist.append("LONG->")
    long_list.clear()
    long_price_list.clear()

def closeshortpositions(exitprice,positions,sign):
    global short_list,df,commission_cost,shortexitloss,shortexitprofit,totalprofit
    
    price=exitprice*2
    commission_cost+=2.5
    print(f'Closing {abs(positions)} short positions | Exit Price:', exitprice,"i:",i)
   

    for j in short_list:
        diff=price-j
        if diff<0:
            shortexitprofit+=abs(diff)
        if diff>0:
            shortexitloss+=diff
    
    x=calcexitpnl(positions,exitprice)
    if x>0:
        sig='Profit'
    else:
        sig='Loss'
    
    keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','PnL','Skew']
    for a in range(1,11):
        keys.append(f'RunningPnL{a}')
    
    values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],0,positions,1,sig,skew]
    for b in range(1,11):
        values.append((calcrunningpnl(positions,exitprice)+0.125)*2)

    if sign==1:
        keys.append('ProfitTarget')
        values.append(1)
    elif sign==-1:
        keys.append('StopLoss')
        values.append(1)
   
    row=dict(zip(keys,values))
    row = {header: row.get(header, '') for header in headers}
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row)
   
    exitpricelist.append([exit_price,"SHORT"])
    profitlist.append("SHORT->")
    short_list.clear()
    short_price_list.clear()
    

def calcexitpnl(p,exitprice):
    global shortexitloss,shortexitprofit,longexitloss,longexitprofit,commission_cost,short_list,long_list
    if p>0:
        pnl=50*((abs(positions)*exitprice)-sum(long_list)+shortexitprofit+longexitprofit-shortexitloss-longexitloss)-commission_cost
    elif p<0:
        pnl=50*(sum(short_list)-(abs(positions)*exitprice)+shortexitprofit+longexitprofit-shortexitloss-longexitloss)-commission_cost
    return pnl-2.5

def calcrunningpnl(p,currentprice):
    global commission_cost,short_price_list,long_price_list
    if p>0 and len(long_price_list)>0:
        runningpnl=(currentprice-(sum(long_price_list)/len(long_price_list)))*50-commission_cost/abs(p)
        return runningpnl
    elif p<0 and len(short_price_list)>0:
        runningpnl=(sum(short_price_list)/len(short_price_list)-currentprice)*50 -commission_cost/abs(p)
        return runningpnl
    else:
        return 0
    
def trackpnl():
    global running_dict
    global df
    max_pnl=max(running_dict.values())
    min_pnl=min(running_dict.values())
    row1={'DateTime':dataset['ts_event'][get_key_by_value(running_dict,max_pnl)],'MaxRunningPnL':max_pnl*20,'Exit':0,'Bid0':dataset['bid_px_00'][get_key_by_value(running_dict,max_pnl)],'Ask0':dataset['ask_px_00'][get_key_by_value(running_dict,max_pnl)],'EntrySignal':0,'Exit':0,'PositionSize':positions,'Skew':dataset['Skew'][get_key_by_value(running_dict,max_pnl)]}
    row2={'DateTime':dataset['ts_event'][get_key_by_value(running_dict,min_pnl)],'MinRunningPnL':min_pnl*20,'Exit':0,'Bid0':dataset['bid_px_00'][get_key_by_value(running_dict,min_pnl)],'Ask0':dataset['ask_px_00'][get_key_by_value(running_dict,min_pnl)],'EntrySignal':0,'Exit':0,'PositionSize':positions,'Skew':dataset['Skew'][get_key_by_value(running_dict,min_pnl)]}
    row1 = {header: row1.get(header, '') for header in headers}
    row2 = {header: row2.get(header, '') for header in headers}
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row1)
        writer.writerow(row2)

    
        
     

def displaystats():
    global totalprofit,profits,losses
    #print("LONG_EXIT_PROFIT:",longexitprofit)
    #print("LONG_EXIT_LOSS:",longexitloss)
    #print("SHORT_EXIT_PROFIT:",shortexitprofit)
    #print("SHORT_EXIT_LOSS:",shortexitloss)
    print("COMMISSION_COST:",commission_cost)
    profit=(longexitprofit+shortexitprofit-longexitloss-shortexitloss)*50-commission_cost
    keys=['DateTime','Bid0','Ask0','EntrySignal','PositionSize','Exit','PnL','PnLValue']
    values=[dataset['ts_event'][i],dataset['bid_px_00'][i],dataset['ask_px_00'][i],0,0,1]

    if profit>0:
        profits+=1
        values.append('Profit')
        values.append(profit)
    else:
        losses+=1
        
        values.append('Loss')
        values.append(profit)
    row=dict(zip(keys,values))
    row = {header: row.get(header, '') for header in headers}
    with open(fileout, 'a', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=headers)
        file.seek(0,2)
        writer.writerow(row)
        
    
    profitlist.append(profit)
    totalprofit+=profit
    print("PROFIT:",profit)
    print("TOTAL_PROFIT:",totalprofit)

def get_key_by_value(dictionary, target_value):
    for key, value in dictionary.items():
        if value == target_value:
            return key

def calcspread(exitprice,positions):
        global long_price_list,short_price_list
        if positions>0:
            return exitprice-long_price_list(len(long_price_list)-1)
        elif positions<0:
            return short_price_list[len(short_price_list)-1]-exitprice
        


for i in range(len(dataset)):
    timenow=dataset['ts_event'][i].time()
    skew = dataset['Skew'][i]
    if timenow>=start_time and timenow <= end_time:

        if positions==0 and commission_cost!=0:  
            displaystats()
            longexitprofit=shortexitprofit=longexitloss=shortexitloss=commission_cost=0
                
                

    
        if abs(positions)==max_positions:
            if start_hold is None:
                start_hold=dataset['ts_event'][i]
                end_hold=start_hold+timedelta(minutes=20)
            
            if interval_start_time is None:
                interval_start_time=dataset['ts_event'][i]
                interval_end_time=interval_start_time+timedelta(minutes=1)

        
            if positions>0 and dataset['ts_event'][i]<end_hold:
                exit_price=dataset['bid_px_00'][i]
                #prof=calcexitpnl(positions,exit_price)
                prof=calcspread(exit_price,positions)
                
                if dataset['ts_event'][i]<=interval_end_time:
                    running_dict[i]=calcrunningpnl(positions,exit_price)

                else:
                    trackpnl()
                    interval_start_time=interval_end_time
                    interval_end_time=interval_start_time+timedelta(minutes=1)
                    running_dict.clear()
                
                if prof>=profit_target:
                    if len(running_dict.values())>0:
                        trackpnl()
                        running_dict.clear()

                    
                    print("RunningPNL while closing:",calcrunningpnl(positions,exit_price))
                    closelongpositions(exit_price,positions,1)
                    positions=0
                    start_hold=end_hold=None
                    interval_start_time=None
                        
                elif prof<0:
                    if abs(prof)>stop_loss:
                        if len(running_dict.values())>0:
                            trackpnl()
                            running_dict.clear()
                        print("STOP LOSS LIMIT REACHED",prof)
                        print("RunningPNL while closing:",calcrunningpnl(positions,exit_price))
                        closelongpositions(exit_price,positions,-1)
                    
                        positions=0
                        running_dict.clear()
                        start_hold=end_hold=None
                        interval_start_time=None

            
            elif positions<0 and dataset['ts_event'][i]<end_hold :   
                
                exit_price=dataset['ask_px_00'][i]
                #prof=calcexitpnl(positions,exit_price)
                prof=calcspread(exit_price,positions)
                
                if dataset['ts_event'][i]<=interval_end_time:
                    running_dict[i]=calcrunningpnl(positions,exit_price)
                
                else:
                    trackpnl()
                    interval_start_time=interval_end_time
                    interval_end_time=interval_start_time+timedelta(minutes=1)
                    running_dict.clear()
                
                
                if prof>=profit_target:
                    if len(running_dict.values())>0:
                        trackpnl()
                        running_dict.clear()

                    print("RunningPNL WHILE CLOSING:",calcrunningpnl(positions,exit_price))
                    closeshortpositions(exit_price,positions,1)
                    
                    positions=0
                    interval_start_time=None
                    start_hold=end_hold=None
                
                elif prof<0:
                    if abs(prof)>stop_loss:

                        if len(running_dict.values())>0:
                            trackpnl()
                            running_dict.clear()

                        print("STOP LOSS LIMIT REACHED",prof)
                        print("RunningPNL while closing:",calcrunningpnl(positions,exit_price))
                        closeshortpositions(exit_price,positions,-1)
                        
                        positions=0
                        interval_start_time=None
                        start_hold=end_hold=None
                    
            elif dataset['ts_event'][i]>=end_hold and positions>0:
                exit_price=dataset['bid_px_00'][i]
                if len(running_dict.values())>0:
                        trackpnl()
                        running_dict.clear()
                    
                print("TIMED OUT, EXITING NOW AT RunningPNL:",calcrunningpnl(positions,exit_price))
                closelongpositions(exit_price,positions,0)
                positions=0
                start_hold=end_hold=None
                interval_start_time=None
            
            elif dataset['ts_event'][i]>=end_hold and positions<0:
                exit_price=dataset['ask_px_00'][i]
                if len(running_dict.values())>0:
                        trackpnl()
                        running_dict.clear()

                print("TIMED OUT, EXITING NOW AT RunningPNL:",calcrunningpnl(positions,exit_price))
                closeshortpositions(exit_price,positions,0)
                positions=0
                start_hold=end_hold=None
                interval_start_time=None
                
                    

        elif skew > k and abs(positions) < max_positions:
            entry_price = dataset['ask_px_00'][i]
            
            if positions>=0:
                long(entry_price)
                
            else:
                exitshort(entry_price)
                
            
        elif skew < -k and abs(positions) < max_positions:
            entry_price = dataset['bid_px_00'][i]
            if positions<=0:
                short(entry_price)
                
            else:
                exitlong(entry_price)
            

    elif timenow>end_time:
        
        
        if positions > 0:
            exit_price = dataset['bid_px_00'][i]
        
            closelongpositions(exit_price,positions,0)
            print("END OF A DAY")
            displaystats()
            positions=0
            longexitprofit=shortexitprofit=longexitloss=shortexitloss=commission_cost=0
            totalprofit=0
            
        elif positions < 0:
            exit_price = dataset['ask_px_00'][i]
        
            closeshortpositions(exit_price,positions,0)
            print("END OF A DAY")
            displaystats()
            positions=0
            longexitprofit=shortexitprofit=longexitloss=shortexitloss=commission_cost=0
            totalprofit=0
        
    elif i ==len(dataset)-1:
        if positions > 0:
            exit_price = dataset['bid_px_00'][i]
            print("SIMULATION TERMINATED")
            closelongpositions(exit_price,positions,0)
            print("END OF DATASET")
            displaystats()
            break
        
        elif positions < 0:
            exit_price = dataset['ask_px_00'][i]
            print("SIMULATION TERMINATED")
            closeshortpositions(exit_price,positions,0)
            print("END OF A DATASET")
            displaystats()
            break



print("PROFIT LIST:")
print(profitlist)
print("NO OF PROFITS:",profits)
print("NUMBER OF LOSSES:",losses)